In [9]:
# import required libriaries
from pathlib import Path
import json

from fpl_predictor.util import *

In [10]:
# path to data directory
#path = Path('/home/jupyter/fpl-prediction/data')
path = Path('./fpl_predictor/data')

In [11]:
# read in latest training set
df_train = pd.read_csv(path/'train_v8.csv', index_col=0, 
                       dtype={'season':str,
                              'comp':str,
                              'squad':str})

# path to current season
season_paths = [path/'2021-22']

# name for current season
season_names = ['2122']

# team codes
teams = pd.read_csv(path/'teams.csv')

# latest remaining season dataset, for latest week's play_proba values
remaining_season = pd.read_csv(path/'remaining_season.csv', index_col=0)

# create players dataset
all_players = build_players(path, season_paths, season_names, teams)

# 2019-20 market value
# table was in slightly different position for 2019-20 season
# should check that 2019-20 season values remain the same once season starts
teams_mv = build_season_mv('2122', 11, range(13,33))

In [12]:
# all_players#[all_players['full_name'] == 'Ben Davies']

In [13]:
df_train.tail(50)

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,xa,sca,gca,passes_completed,passes,passes_pct,carries,dribbles_completed,dribbles,crowds
116448,Teemu Pukki,3,4,72,Norwich,Leicester City,0.421101,1.222091,True,7,...,0.1,1.0,0.0,8.0,12.0,66.7,8.0,0.0,0.0,1
116449,Theo Walcott,3,3,0,Southampton,Newcastle United,0.536067,0.539622,False,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
116450,Thiago Alcántara do Nascimento,3,3,16,Liverpool,Chelsea,1.953880,1.960545,True,1,...,0.0,2.0,0.0,15.0,19.0,78.9,15.0,0.0,0.0,1
116451,Thiago Emiliano da Silva,3,2,45,Chelsea,Liverpool,1.960545,1.953880,False,1,...,0.0,0.0,0.0,12.0,14.0,85.7,10.0,0.0,0.0,1
116452,Thomas Partey,3,3,0,Arsenal,Manchester City,1.218537,2.310444,False,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
116453,Timothy Castagne,3,2,79,Leicester City,Norwich,1.222091,0.421101,False,2,...,0.0,0.0,0.0,43.0,50.0,86.0,28.0,0.0,0.0,1
116454,Timo Werner,3,4,0,Chelsea,Liverpool,1.960545,1.953880,False,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
116455,Tim Krul,3,1,90,Norwich,Leicester City,0.421101,1.222091,True,1,...,0.0,0.0,0.0,24.0,35.0,68.6,18.0,0.0,0.0,1
116456,Tino Livramento,3,2,90,Southampton,Newcastle United,0.536067,0.539622,False,1,...,0.9,4.0,0.0,41.0,52.0,78.8,39.0,1.0,3.0,1
116457,Toby Alderweireld,3,2,0,Tottenham Hotspur,Watford,1.537334,0.295470,True,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [14]:
# find the latest gameweek
last_gw = df_train['gw'][df_train['season'] == season_names[0]].max()

if np.isnan(last_gw): 
    last_gw = 1 
else: 
    last_gw = last_gw + 1
    
last_gw

4

In [15]:
# build training set for latest gameweek
df_latest = build_season(season_paths[0], season_names[0], all_players, teams, teams_mv, [last_gw])

In [16]:
df_latest.drop_duplicates(inplace=True)

In [17]:
df_latest.head(50)

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,selected,team_a_score,team_h_score,threat,transfers_balance,transfers_in,transfers_out,yellow_cards,kickoff_time,season
0,Aaron Connolly,4,4,0,Brighton and Hove Albion,Brentford,0.550960,0.369638,False,0,...,12065,1,0,0.0,-3682,1857,5539,0,2021-09-11T14:00:00Z,2122
1,Aaron Cresswell,4,2,90,West Ham United,Southampton,0.787799,0.535859,False,8,...,1324980,0,0,9.0,-94228,102407,196635,0,2021-09-11T14:00:00Z,2122
2,Aaron Lennon,4,3,14,Burnley,Everton,0.322670,1.025416,False,1,...,590,1,3,0.0,271,382,111,0,2021-09-13T19:00:00Z,2122
3,Aaron Ramsdale,4,1,90,Arsenal,Norwich,1.218063,0.420937,True,6,...,17374,0,1,0.0,6113,10225,4112,0,2021-09-11T14:00:00Z,2122
4,Aaron Ramsey,4,3,0,Aston Villa,Chelsea,0.906719,1.959783,False,0,...,15295,0,3,0.0,891,3516,2625,0,2021-09-11T16:30:00Z,2122
5,Aaron Wan-Bissaka,4,2,90,Manchester United,Newcastle United,2.081367,0.536081,True,2,...,618617,1,4,4.0,-55262,37244,92506,0,2021-09-11T14:00:00Z,2122
6,Abdoulaye Doucouré,4,3,90,Everton,Burnley,1.025416,0.322670,True,10,...,141987,1,3,21.0,9162,45662,36500,0,2021-09-13T19:00:00Z,2122
7,Adama Traoré,4,3,80,Wolverhampton Wanderers,Watford,0.868522,0.295355,False,3,...,319053,2,0,8.0,197962,303693,105731,0,2021-09-11T14:00:00Z,2122
8,Adam Armstrong,4,4,73,Southampton,West Ham United,0.535859,0.787799,True,2,...,275824,0,0,21.0,-7420,99289,106709,0,2021-09-11T14:00:00Z,2122
9,Adam Forshaw,4,3,0,Leeds,Liverpool,0.556956,1.953121,True,0,...,1227,3,0,0.0,183,401,218,0,2021-09-12T15:30:00Z,2122


In [18]:
df_latest.shape

(599, 34)

In [19]:
last_play_proba = remaining_season[remaining_season['gw'] == last_gw][['player', 'team', 'play_proba']]

In [20]:
last_play_proba.head()

,player,team,play_proba
0,Conor Gallagher,Crystal Palace,1.0
1,Vicente Guaita,Crystal Palace,1.0
2,James Tomkins,Crystal Palace,1.0
3,James McArthur,Crystal Palace,1.0
4,Christian Benteke,Crystal Palace,1.0


In [21]:
# add latest week's play_proba for each player
df_latest = df_latest.merge(last_play_proba, on=['player', 'team'], how='left')

In [22]:
df_latest.drop_duplicates(inplace=True)

In [23]:
df_train.drop_duplicates(inplace=True)

In [24]:
if last_gw==1:
    df_latest['relative_market_value_team_season'] = df_latest['relative_market_value_team']
    df_latest['relative_market_value_opponent_team_season'] = df_latest['relative_market_value_opponent_team']

In [25]:
# for when you're doing the season value properly
# relative_market_value_season = train_df[train_df['season'] == '2021'][['team', 'relative_market_value_team']].groupby(['team']).first()
# relative_market_value_season = relative_market_value_season.rename(columns={'relative_market_value_team': 'season_value'})
# relative_market_value_season

In [26]:
# now get futher stats from fbref
# standard stats
features = ['squad', 'comp', 'round', 'shots_total', 'shots_on_target', 'touches', 'pressures',
            'tackles', 'interceptions', 'blocks', 'xg', 'npxg', 'xa', 'sca', 
            'gca', 'passes_completed', 'passes', 'passes_pct',  'carries', 
            'dribbles_completed', 'dribbles']

In [27]:
# url for the premier league
url_base = 'https://fbref.com'
pl_history_page = '/en/comps/9/history/Premier-League-Seasons'
url = url_base + pl_history_page

# scrape the history table to get latest season
pl_history_table = get_table(url)
pl_history_urls = get_urls(['season'], 'season', pl_history_table, url_base)
season_urls = pl_history_urls[0:1]

df_enhanced = get_stats(features, season_urls, url_base)

2021-2022
https://fbref.com/en/comps/9/Premier-League-Stats
Manchester Utd
https://fbref.com/en/squads/19538871/Manchester-United-Stats
David de Gea
Harry Maguire
Luke Shaw
Aaron Wan-Bissaka
Bruno Fernandes
Mason Greenwood
Paul Pogba
Fred
Nemanja Matić
Jadon Sancho
Victor Lindelöf
Raphaël Varane
Daniel James
Anthony Martial
Cristiano Ronaldo
Scott McTominay
Edinson Cavani
Jesse Lingard
Donny van de Beek
Diogo Dalot
Chelsea
https://fbref.com/en/squads/cff3d9bb/Chelsea-Stats
Edouard Mendy
Antonio Rüdiger
Marcos Alonso
Jorginho
Mateo Kovačić
César Azpilicueta
Andreas Christensen
Romelu Lukaku
Mason Mount
Kai Havertz
Trevoh Chalobah
Reece James
Thiago Silva
Timo Werner
Hakim Ziyech
Callum Hudson-Odoi
Christian Pulisic
N'Golo Kanté
Saúl Ñíguez
Emerson Palmieri
Liverpool
https://fbref.com/en/squads/822bd0ba/Liverpool-Stats
Trent Alexander-Arnold
Alisson
Virgil van Dijk
Sadio Mané
Joël Matip
Mohamed Salah
Diogo Jota
Harvey Elliot
Fabinho
Jordan Henderson
Kostas Tsimikas
Andrew Robertson
Naby 

In [28]:
# get the player name matching dictionary
with open(path/'match_list.json') as f:
  match_list = json.load(f)

match_list

{'Fabinho': 'Fabio Henrique Tavares',
 'Alisson': 'Alisson Ramses Becker',
 'Joël Matip': 'Joel Matip',
 'Joe Gomez': 'Joseph Gomez',
 'Naby Keïta': 'Naby Keita',
 'Adrián': 'Adrián San Miguel del Castillo',
 'Thiago Alcántara': 'Thiago Alcántara do Nascimento',
 'Allan': 'Allan Marques Loureiro',
 'Richarlison': 'Richarlison de Andrade',
 'Gylfi Sigurðsson': 'Gylfi Sigurdsson',
 'Séamus Coleman': 'Seamus Coleman',
 'André Gomes': 'André Filipe Tavares Gomes',
 'Bernard': 'Bernard Anício Caldeira Duarte',
 'Çağlar Söyüncü': 'Çaglar Söyüncü',
 'Bruno Fernandes': 'Bruno Miguel Borges Fernandes',
 'Fred': 'Frederico Rodrigues de Paula Santos',
 'Nemanja Matić': 'Nemanja Matic',
 'Alex Telles': 'Alex Nicolao Telles',
 'Pierre Højbjerg': 'Pierre-Emile Højbjerg',
 'Son Heung-min': 'Heung-Min Son',
 'Lucas Moura': 'Lucas Rodrigues Moura da Silva',
 'Érik Lamela': 'Erik Lamela',
 'Dele Alli': 'Bamidele Alli',
 'Carlos Vinícius': 'Carlos Vinicius Alves Morais',
 'Ederson': 'Ederson Santana de M

In [29]:
team_list = {}
team_list['Brighton'] = 'Brighton and Hove Albion'
team_list['Leeds United'] = 'Leeds'
team_list['Manchester Utd'] = 'Manchester United'
team_list['Newcastle Utd'] = 'Newcastle United'
team_list['Norwich City'] = 'Norwich'
team_list['Sheffield Utd'] = 'Sheffield United'
team_list['Tottenham'] = 'Tottenham Hotspur'
team_list['West Brom'] = 'West Bromwich Albion'
team_list['West Ham'] = 'West Ham United'
team_list['Wolves'] = 'Wolverhampton Wanderers'

In [30]:
# replace non-matching names/teams and extract gw as integer
df_enhanced['alt_name'] = df_enhanced['player'].replace(match_list)
df_enhanced['squad'] = df_enhanced['squad'].replace(team_list)
df_enhanced['round'] = df_enhanced['round'].str[-2:].astype('int')

In [31]:
# remove duplicated (happens when players transferred within season)
df_enhanced = df_enhanced.drop_duplicates()

In [32]:
df_enhanced

,season,player,date,squad,comp,round,shots_total,shots_on_target,touches,pressures,...,xa,sca,gca,passes_completed,passes,passes_pct,carries,dribbles_completed,dribbles,alt_name
0,2021-2022,David de Gea,2021-08-14,Manchester United,Premier League,1,0,0,36,0,...,0.0,0,0,30,35,85.7,18,0,0,David de Gea
1,2021-2022,David de Gea,2021-08-22,Manchester United,Premier League,2,0,0,22,0,...,0.0,0,0,17,18,94.4,9,0,0,David de Gea
2,2021-2022,David de Gea,2021-08-29,Manchester United,Premier League,3,0,0,24,0,...,0.0,0,0,17,20,85.0,5,0,0,David de Gea
3,2021-2022,David de Gea,2021-09-11,Manchester United,Premier League,4,0,0,14,0,...,0.0,1,0,12,13,92.3,2,0,0,David de Gea
4,2021-2022,Harry Maguire,2021-08-14,Manchester United,Premier League,1,1,0,56,4,...,0.0,0,0,39,45,86.7,36,0,0,Harry Maguire
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1104,2021-2022,Josh Sargent,2021-08-28,Norwich,Premier League,3,1,0,6,1,...,0.1,1,0,3,5,60.0,4,0,0,Joshua Sargent
1105,2021-2022,Bali Mumba,2021-08-21,Norwich,Premier League,2,0,0,19,12,...,0.0,0,0,7,10,70.0,9,0,1,Bali Mumba
1106,2021-2022,Adam Idah,2021-08-14,Norwich,Premier League,1,1,1,6,5,...,0.0,0,0,3,5,60.0,2,0,0,Adam Idah
1107,2021-2022,Adam Idah,2021-08-28,Norwich,Premier League,3,1,0,6,2,...,0.0,0,0,4,4,100.0,5,0,1,Adam Idah


In [33]:
# create date field in same format as enhanced data
df_latest['date'] = df_latest['kickoff_time'].str[:10]

In [34]:
# check if any names aren't matching
enhanced_players = df_enhanced[df_enhanced['date'].isin(df_latest['date'].unique())]['alt_name'].unique().tolist()
standard_players = df_latest['player'].unique().tolist()

unmatched_players = [x for x in enhanced_players if x not in standard_players]
unmatched_players

['Cristiano Ronaldo',
 'Emerson',
 'Mads Roerslev',
 'Odsonne Édouard',
 'Hwang Hee-chan']

In [35]:
# check if any teams aren't matching
enhanced_teams = df_enhanced[df_enhanced['date'].isin(df_latest['date'].unique())]['squad'].unique().tolist()
standard_teams = df_latest['team'].unique().tolist()

unmatched_teams = [x for x in enhanced_teams if x not in standard_teams]
unmatched_teams

[]

In [36]:
# any additional corrections
# match_list['Cristiano Ronaldo'] = 'Cristiano Ronaldo dos Santos Aveiro'
# match_list['Emerson'] = 'Emerson Aparecido Leite de Souza Junior'
# match_list['Mads Roerslev'] = 'Mads Roerslev Rasmussen'
# match_list['Odsonne Édouard'] = 'Odsonne Edouard'
# match_list['Hwang Hee-chan'] = 'Hee-Chan Hwang'

In [37]:
# replace additional non-matching names
# df_enhanced['alt_name'] = df_enhanced['player'].replace(match_list)

In [38]:
# make sure there are no non matches after update
# enhanced_players =  df_enhanced[df_enhanced['round'] == last_gw]['alt_name'].unique().tolist()
# standard_players = df_latest['player'].unique().tolist()

# unmatched_players = [x for x in enhanced_players if x not in standard_players]
# unmatched_players

[]

In [39]:
# if match list was updated then update the file
# json_file = json.dumps(match_list)
# f = open(path/"match_list.json","w")
# f.write(json_file)
# f.close()

In [40]:
# check there are no duplicates in the join fields for the enhanced data
assert(len(df_enhanced[df_enhanced.duplicated(subset=['alt_name', 'date'], keep=False)]) == 0)

In [41]:
df_latest[df_latest.duplicated(subset=['date','player','team'], keep=False)]

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,team_h_score,threat,transfers_balance,transfers_in,transfers_out,yellow_cards,kickoff_time,season,play_proba,date


In [42]:
# check there are no duplicates in the join fields for the standard data
assert(len(df_train[df_train.duplicated(subset=['date','player','team'])]) == 0)

In [43]:
# df_train[df_train.duplicated(subset=['date','player','team'])]

In [44]:
# join standard and enhanced dataframes
df_latest_further = df_latest.merge(df_enhanced.drop(['player', 'season', 'round'], axis=1), 
                                   left_on=['date', 'player', 'team'], 
                                   right_on=['date', 'alt_name', 'squad'], how='left')

In [45]:
# drop the alt name field and check the length of the standard
# dataframe has not increased
df_latest_further = df_latest_further.drop(['alt_name'], axis=1)
assert(len(df_latest_further) == len(df_latest_further))

In [46]:
# add crowds field
df_latest_further['crowds'] = 1

In [47]:
assert(sum(df_latest_further[df_latest_further['minutes'] > 0]['xg'].isnull()) == 0)

In [48]:
# df_latest_further[df_latest_further['minutes'] > 0][df_latest_further[df_latest_further['minutes'] > 0]['xg'].isnull()]

In [49]:
df_train_new = pd.concat([df_train, df_latest_further], ignore_index=True, axis=0, sort=False)

In [50]:
df_train_new[df_train_new['player'] == 'Harry Kane'].tail()

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,xa,sca,gca,passes_completed,passes,passes_pct,carries,dribbles_completed,dribbles,crowds
114301,Harry Kane,38,4,90,Tottenham Hotspur,Leicester City,1.568259,1.165630,False,10,...,0.1,3,2,8,16,50,28,6,7,1
114999,Harry Kane,1,4,0,Tottenham Hotspur,Manchester City,1.560569,2.349720,True,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
115556,Harry Kane,2,4,18,Tottenham Hotspur,Wolverhampton Wanderers,1.558729,0.858519,False,0,...,0.1,2,0,7,10,70,7,0,0,1
116128,Harry Kane,3,4,90,Tottenham Hotspur,Watford,1.537334,0.295470,True,1,...,0.3,3,0,21,26,80.8,30,2,3,1
116711,Harry Kane,4,4,90,Tottenham Hotspur,Crystal Palace,1.547840,0.531751,False,2,...,0.0,1,0,16,23,69.6,26,2,3,1


In [51]:
df_train_new.tail(7)

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,xa,sca,gca,passes_completed,passes,passes_pct,carries,dribbles_completed,dribbles,crowds
117090,Yerson Mosquera Valdelamar,4,2,0,Wolverhampton Wanderers,Watford,0.868522,0.295355,False,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
117091,Yoane Wissa,4,3,0,Brentford,Brighton and Hove Albion,0.369638,0.550960,True,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
117092,Youri Tielemans,4,3,90,Leicester City,Manchester City,1.221616,2.309546,True,2,...,0.0,0,0,35,42,83.3,32,0,0,1
117093,Yves Bissouma,4,3,90,Brighton and Hove Albion,Brentford,0.550960,0.369638,False,3,...,0.0,0,0,52,64,81.3,53,2,3,1
117094,Zack Steffen,4,1,0,Manchester City,Leicester City,2.309546,1.221616,False,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
117095,Álvaro Fernández,4,1,0,Brentford,Brighton and Hove Albion,0.369638,0.550960,True,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
117096,Çaglar Söyüncü,4,2,90,Leicester City,Manchester City,1.221616,2.309546,True,1,...,0.0,0,0,53,62,85.5,46,0,0,1


In [52]:
df_latest_further.shape

(599, 57)

In [53]:
df_train_new.shape

(117097, 59)

In [54]:
# save latest training set to csv
# overwrite existing one
df_train_new.to_csv(path/'train_v8.csv')

In [55]:
# get fixtures
fixtures = pd.read_csv(path/'2021-22/fixtures.csv')
team_dict = teams[['team_2122', 'team']].dropna().set_index('team_2122').to_dict()['team']

fixtures['home_team'] = fixtures['team_h'].replace(team_dict)
fixtures['away_team'] = fixtures['team_a'].replace(team_dict)
fixtures = fixtures[['home_team', 'away_team', 'kickoff_time', 'event']]
fixtures = fixtures.rename(columns={'event':'gw'})


# use these two lines if updating play_proba later in week
# current_gw = last_gw
# df_train_new = pd.read_csv(path/'train_v7.csv', index_col=0, dtype={'season':str,
#                                                                     'comp':str,
#                                                                     'squad':str})

# set starting gameweek (where are we right now in the season)
current_gw = last_gw + 1
fixtures = fixtures[fixtures['gw'] >= current_gw].sort_values(by=['gw'])
fixtures['match_no'] = np.arange(fixtures.shape[0])

In [56]:
fixtures.head(50)

,home_team,away_team,kickoff_time,gw,match_no
40,Newcastle United,Leeds,2021-09-17T19:00:00Z,5,0
41,Wolverhampton Wanderers,Brentford,2021-09-18T11:30:00Z,5,1
42,Burnley,Arsenal,2021-09-18T14:00:00Z,5,2
43,Liverpool,Crystal Palace,2021-09-18T14:00:00Z,5,3
44,Manchester City,Southampton,2021-09-18T14:00:00Z,5,4
45,Norwich,Watford,2021-09-18T14:00:00Z,5,5
46,Aston Villa,Everton,2021-09-18T16:30:00Z,5,6
47,Brighton and Hove Albion,Leicester City,2021-09-19T13:00:00Z,5,7
48,West Ham United,Manchester United,2021-09-19T13:00:00Z,5,8
49,Tottenham Hotspur,Chelsea,2021-09-19T15:30:00Z,5,9


In [57]:
# add universal team codes for home and away teams
fixtures = fixtures.merge(teams, left_on='home_team', right_on='team', how='left')
fixtures = fixtures.merge(teams, left_on='away_team', right_on='team', how='left')
fixtures = fixtures[['gw', 'match_no', 'home_team', 'away_team', 'kickoff_time', 'team_code_x', 'team_code_y']]
fixtures.rename(index=str,
                columns={'team_code_x':'home_team_code',
                         'team_code_y':'away_team_code'},
                inplace=True)

In [58]:
fixtures.head()

,gw,match_no,home_team,away_team,kickoff_time,home_team_code,away_team_code
0,5,0,Newcastle United,Leeds,2021-09-17T19:00:00Z,4,2
1,5,1,Wolverhampton Wanderers,Brentford,2021-09-18T11:30:00Z,39,94
2,5,2,Burnley,Arsenal,2021-09-18T14:00:00Z,90,3
3,5,3,Liverpool,Crystal Palace,2021-09-18T14:00:00Z,14,31
4,5,4,Manchester City,Southampton,2021-09-18T14:00:00Z,43,20


In [59]:
# all_players.replace(to_replace='None', value=1, inplace=True)

In [60]:
all_players[all_players['play_proba_2122'] == '0']

,full_name,play_proba_2122,position_2122,id_2122,cost_2122,team_2122,web_name_2122
1,Rúnar Alex Rúnarsson,0,1,2,40,3,Rúnarsson
2,Willian Borges Da Silva,0,3,3,63,3,Willian
6,Granit Xhaka,0,3,7,49,3,Xhaka
8,Héctor Bellerín,0,2,9,48,3,Bellerín
11,Mohamed Naser El Sayed Elneny,0,3,12,45,3,Elneny
...,...,...,...,...,...,...,...
584,Pedro Lomba Neto,0,3,441,64,39,Neto
585,Owen Otasowie,0,3,442,45,39,Otasowie
587,Andreas Söndergaard,0,1,444,40,39,Söndergaard
594,Patrick Cutrone,0,4,523,50,39,Cutrone


In [61]:
# join home team to all players for current season
home_df = fixtures.merge(all_players, 
               left_on='home_team_code', 
               right_on='team_2122', 
               how='left')

# pull out the required fields and rename columns
home_df = home_df[['gw', 'match_no', 'home_team', 'away_team', 'kickoff_time', 
                   'full_name', 'position_2122', 'cost_2122', 'play_proba_2122', 
                   'web_name_2122']]
home_df.rename(index=str, 
               columns={'home_team':'team',
                        'away_team':'opponent_team',
                        'full_name':'player',
                        'position_2122':'position',
                        'cost_2122':'price',
                        'play_proba_2122':'play_proba',
                        'web_name_2122':'short_name'},
              inplace=True)

# add home flag
home_df['was_home'] = True

In [62]:
# join away team to all players for current season
away_df = fixtures.merge(all_players, 
               left_on='away_team_code', 
               right_on='team_2122', 
               how='left')

# pull out the required fields and rename columns
away_df = away_df[['gw',  'match_no', 'away_team', 'home_team', 'kickoff_time', 
                   'full_name', 'position_2122', 'cost_2122', 'play_proba_2122', 
                   'web_name_2122']]
away_df.rename(index=str, 
               columns={'away_team':'team',
                        'home_team':'opponent_team',
                        'full_name':'player',
                        'position_2122':'position',
                        'cost_2122':'price',
                        'play_proba_2122':'play_proba',
                        'web_name_2122':'short_name'},
              inplace=True)

# add home flag
away_df['was_home'] = False

In [63]:
# look at away players
away_df.head()

,gw,match_no,team,opponent_team,kickoff_time,player,position,price,play_proba,short_name,was_home
0,5,0,Leeds,Newcastle United,2021-09-17T19:00:00Z,Pablo Hernández Domínguez,3,55,0,Hernández,False
1,5,0,Leeds,Newcastle United,2021-09-17T19:00:00Z,Francisco Casilla Cortés,1,45,0,Casilla,False
2,5,0,Leeds,Newcastle United,2021-09-17T19:00:00Z,Liam Cooper,2,45,None,Cooper,False
3,5,0,Leeds,Newcastle United,2021-09-17T19:00:00Z,Luke Ayling,2,45,None,Ayling,False
4,5,0,Leeds,Newcastle United,2021-09-17T19:00:00Z,Mateusz Klich,3,55,100,Klich,False


In [64]:
# look at home players
home_df.head()

,gw,match_no,team,opponent_team,kickoff_time,player,position,price,play_proba,short_name,was_home
0,5,0,Newcastle United,Leeds,2021-09-17T19:00:00Z,Joseph Willock,3,60,100,Willock,True
1,5,0,Newcastle United,Leeds,2021-09-17T19:00:00Z,Jonjo Shelvey,3,54,0,Shelvey,True
2,5,0,Newcastle United,Leeds,2021-09-17T19:00:00Z,Matt Ritchie,2,50,None,Ritchie,True
3,5,0,Newcastle United,Leeds,2021-09-17T19:00:00Z,Ciaran Clark,2,44,100,Clark,True
4,5,0,Newcastle United,Leeds,2021-09-17T19:00:00Z,Karl Darlow,1,45,100,Darlow,True


In [65]:
# # calculate average minutes over last 5 gameweeks
# # first get last 5 gameweeks in current season
# recent_mins = df_train_new[(df_train_new['gw'] >= current_gw - 5) & 
#                            (df_train_new['season'] == season_names[0])][['player', 'minutes', 'play_proba']]

# # recent_mins = df_train_new[((df_train_new['gw'] >= 40 - 5) & 
# #                            (df_train_new['season'] == '1920')) | 
# #                            ((df_train_new['gw'] >= 1) & 
# #                            (df_train_new['season'] == '2021'))][['player', 'minutes', 'play_proba']]

# # exclude weeks where the player had an injury designation (i.e. keep where play_proba is 1)
# recent_mins = recent_mins[recent_mins['play_proba'] == 1]

# len(recent_mins)

# # average for each player
# recent_mins = recent_mins.groupby(['player'])['minutes'].mean().to_dict()

# recent_mins

In [66]:
# concatenate home and away players
remaining_season_df = home_df.append(away_df).reset_index(drop=True)

# add home and away team market values
remaining_season_df = remaining_season_df.merge(teams_mv[teams_mv['season'] == '2122'],
                                                left_on='team', 
                                                right_on='name', 
                                                how='left').drop(['name', 'season'], axis=1)

remaining_season_df = remaining_season_df.merge(teams_mv[teams_mv['season'] == '2122'],
                                                left_on='opponent_team', 
                                                right_on='name', 
                                                how='left').drop(['name', 'season'], axis=1)

remaining_season_df.rename(index=str, 
                           columns={'relative_market_value_x':'relative_market_value_team',
                                    'relative_market_value_y':'relative_market_value_opponent_team'},
                           inplace=True)

# add season name
remaining_season_df['season'] = season_names[0]

# divide cost by 10 for actual cost
remaining_season_df['price'] = remaining_season_df['price']/10

# set availability probability
# 0 = 0% chance, 25 = 25% chance, etc
# 'None' or '100' = 100% chance
remaining_season_df.loc[remaining_season_df['play_proba'] == 'None', 'play_proba'] = 100
remaining_season_df['play_proba'] = remaining_season_df['play_proba'].astype('float') / 100

# cast position to integer
remaining_season_df['position'] = remaining_season_df['position'].astype(int)

# # add minutes based on last five fixtures
# remaining_season_df['minutes'] = remaining_season_df['player'].map(recent_mins)
# remaining_season_df['minutes'].fillna(0, inplace=True)

# # multiply minutes by play probability
# remaining_season_df['minutes'] = remaining_season_df['minutes'] * remaining_season_df['play_proba']

In [67]:
remaining_season_df.head(50)

,gw,match_no,team,opponent_team,kickoff_time,player,position,price,play_proba,short_name,was_home,relative_market_value_team,relative_market_value_opponent_team,season
0,5,0,Newcastle United,Leeds,2021-09-17T19:00:00Z,Joseph Willock,3,6.0,1.0,Willock,True,0.536081,0.556956,2122
1,5,0,Newcastle United,Leeds,2021-09-17T19:00:00Z,Jonjo Shelvey,3,5.4,0.0,Shelvey,True,0.536081,0.556956,2122
2,5,0,Newcastle United,Leeds,2021-09-17T19:00:00Z,Matt Ritchie,2,5.0,1.0,Ritchie,True,0.536081,0.556956,2122
3,5,0,Newcastle United,Leeds,2021-09-17T19:00:00Z,Ciaran Clark,2,4.4,1.0,Clark,True,0.536081,0.556956,2122
4,5,0,Newcastle United,Leeds,2021-09-17T19:00:00Z,Karl Darlow,1,4.5,1.0,Darlow,True,0.536081,0.556956,2122
5,5,0,Newcastle United,Leeds,2021-09-17T19:00:00Z,Martin Dubravka,1,4.5,0.0,Dubravka,True,0.536081,0.556956,2122
6,5,0,Newcastle United,Leeds,2021-09-17T19:00:00Z,Callum Wilson,4,7.4,0.0,Wilson,True,0.536081,0.556956,2122
7,5,0,Newcastle United,Leeds,2021-09-17T19:00:00Z,Jeff Hendrick,3,4.9,1.0,Hendrick,True,0.536081,0.556956,2122
8,5,0,Newcastle United,Leeds,2021-09-17T19:00:00Z,Ryan Fraser,3,5.4,1.0,Fraser,True,0.536081,0.556956,2122
9,5,0,Newcastle United,Leeds,2021-09-17T19:00:00Z,Jamaal Lascelles,2,4.4,1.0,Lascelles,True,0.536081,0.556956,2122


In [68]:
# save latest prediction set to csv
remaining_season_df.to_csv(path/'remaining_season.csv')